In [9]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import traceback as tb
import json
# from pymongo import MongoClient

# In[83]:

###上層


url = 'https://ifoodie.tw/search'


In [10]:


driver = webdriver.PhantomJS(executable_path=r'C:\Users\user\Desktop\phantomjs-2.1.1-windows\bin\phantomjs.exe')  # PhantomJS
driver.get(url)  # 把網址交給瀏覽器 
pageSource = driver.page_source  # 取得網頁原始碼
soup = BeautifulSoup(pageSource, 'lxml')  # 解析器接手

In [11]:




def page_crawler(no_page):
    try:
        url1 = 'https://ifoodie.tw'
        for i in range(1,no_page+1,1):#一頁有20家爬蟲 這個例子應該有200家
            for tag in soup.select('div.blog-item-s.ng-scope > div.media > div.media-body > div.title.media-heading > a'):
                url_next = url1 + tag['href']
                print(url_next)
                next_crawler(url_next)#呼叫下一層的爬蟲
    except IndexError:
        print("finding error")
#====================================================================================================================

In [12]:
def next_crawler(url_next):
    ### 中層   

    try:
    # Example:茶寮侘助,餐廳外部資訊,有些東西是我們所要的
#     url = 'https://ifoodie.tw/blog/58bcd12b699b6e645195f4f3-漂丿燒肉食堂'這行網址是練習用的,check時可忽略
        res = requests.get(url_next)
        soup = BeautifulSoup(res.text, 'lxml')

        mydict={}
        #熱門指數
        hot_soup = soup.select("#ratingStar > span")
        for hot in hot_soup:
            熱門指數 = hot.text.strip()
        mydict['熱門指數'] = 熱門指數

        #餐廳評分
        r_rate = soup.select("span.rating.rating4")
        mydict['rating'] = r_rate[0].text

        #作者
        author_name = soup.select("a.name")
        author_name = author_name[0].text
        mydict['author_name'] = author_name

        #作者 url
        author_url = soup.find("a", {"class":"name"})
        author_url = author_url.get("href")
        mydict['author_url'] = author_url
        
        #照片
        img_url = soup.select(".pull-left > .media-object.cover")
        img_url = img_url[0].get("style").split("(")[1].split(")")[0]
        mydict['img_url'] = img_url
#         print(img_url)
        
        # 標籤 再做細部處理==>去掉旁邊的html標籤
        tag_list=[]
        tags = soup.find_all("a", {"rel":"tag"})
        for tag in tags:
            tag = tag.text
            tag_list.append(tag)
            mydict['tag']=tag_list

        #觀看人數
        see_soup = soup.select("div.stat")
        see_soup = see_soup[0].text
        mydict['see']=see_soup

        #餐廳電話號碼
        phone_number = soup.select("div.phone.right")[0].text.split(":")[1]
        phone_number
        mydict['phone_number'] = phone_number
        #超連結
        url = soup.select("div.restaurant.item.right > h4 > a")
        url= url[0].get('href')
        

        url_final = 'https://ifoodie.tw{}'.format(url)
        mydict['web_url'] = url_final

        final_crawler(url_final,mydict)        
    except IndexError:
        pass

In [13]:
def final_crawler(url_final,mydict):
    # Example:茶寮侘助,餐廳外部資訊,有些東西是我們所要的
    try:
        res = requests.get(url_final)
        soup = BeautifulSoup(res.text, 'lxml')


        #營業地址
        #mydict={}
        reup = soup.select("span.info_detail")
        #print(reup)
        mydict['address']=reup[0].text
            #營業時間
        mydict['opentime']=reup[1].text
            #均消
        mydict['avg_consume']=reup[2].text

            #餐廳名字
        restaurant_name = soup.select("h1.title")[0]
        restaurant_name=restaurant_name.text
        mydict['restaurant_name']=restaurant_name
            #餐廳分類
        reup1 = soup.select("a.info_detail")
        #print(reup1)
        for tag1 in reup1:
            tag1=tag1.text
            mydict['category']=tag1

        #訪客言談 細部處理 去掉旁邊的html標籤
        message_total=[]
        #人
        message_mans=soup.select('div.right_wrap > div.user_name > a')
        for message_man in message_mans:
            mydict['message_man']=message_man.text
           
        #時間
#         message_times=soup.select('div.info > meta')
#         for message_time in message_times:
#             mydict['message_time']=message_time.text
            #print(message_time)
        #留言
        message_list=[]
        messages = soup.select("div.message > div")
        for message in messages:
            message = message.text
            message_list.append(message)
            mydict['message'] = message_list

            
#         tag_list=[]
#         tags = soup.find_all("a", {"rel":"tag"})
#         for tag in tags:
#             tag = tag.text
#             tag_list.append(tag)
#             mydict['tag']=tag_list
        print(mydict)
#         goto_mongo(mydict) 
            
        
        
        
    except IndexError:
        print('sorry not catch the information')
 


    

In [14]:
page_crawler(1)

https://ifoodie.tw/blog/58c15782cc21c4893d0a3905-市民大道燒烤店，原木 燒烤.海鮮，台北吃宵夜小酌聊天的燒烤屋推薦
{'avg_consume': '今日營業: 17:30-02:00', 'author_name': '陳小可', 'tag': ['原木 燒烤.海鮮'], '熱門指數': '5', 'category': '燒烤類', 'opentime': '0227213331', 'phone_number': ' 02-2721-3331', 'address': '臺北市松山區市民大道四段75號', 'web_url': 'https://ifoodie.tw/restaurant/58c198fa2756dd732689b6f8', 'message_man': '陳小可', 'see': ' 觀看: 71', 'rating': '4.0', 'author_url': 'https://ifoodie.tw/user/54637e1cd4fdab447a817870', 'img_url': 'https://farm4.staticflickr.com/3831/32326043104_f7d70d82dd_c.jpg', 'restaurant_name': '原木燒烤海鮮', 'message': ['市民大道燒烤店，原木 燒烤.海鮮，台北吃宵夜小酌聊天的燒烤屋推薦']}
https://ifoodie.tw/blog/58c158ca850996503e8be87a-新莊CP值超高平價好吃牛排店-飄也香牛排館！海陸雙拼只要190元，三拼只要240元，超便宜超划算吃超飽！還有附玉米濃湯、飲料、冰淇淋吃到飽！近新莊高中、新莊體育館！(新莊美食推薦)(文末抽獎)
{'avg_consume': '尚無營業時間資訊', 'author_name': '猴屁的異想世界', 'tag': ['飄也香牛排館', '新莊美食推薦', '新莊餐廳推薦', '新莊牛排推薦', '新莊好吃牛排', '飄也香牛排館電話', '飄也香牛排館地址', '飄也香牛排館營業時間', '新莊體育館', '新莊體育館美食', '新莊高中美食', '新莊吃到飽', '新莊便宜牛排', '新莊好吃雞排', '新莊美食'], '熱門指

In [7]:
# def goto_mongo(mydict):#要開啟 mongo的server能夠使用
#     client = MongoClient()
#     db= client['test']
#     collect = db['test_ifood']
#     #一次insert一整個list
#     collect.insert_one(mydict)

In [256]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import traceback as tb
import json

    # Example:茶寮侘助,餐廳外部資訊,有些東西是我們所要的
url = 'https://ifoodie.tw/blog/58bcd12b699b6e645195f4f3-漂丿燒肉食堂'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

mydict={}
    #熱門指數
hot_soup = soup.select("#ratingStar > span")
for hot in hot_soup:
    熱門指數 = hot.text.strip()
mydict['熱門指數'] = 熱門指數

    #餐廳評分
r_rate = soup.select("span.rating.rating4")
mydict['rating'] = r_rate[0].text
    #圖片
img_url = soup.select(".pull-left > .media-object.cover")
img_url = img_url[0].get("style").split("(")[1].split(")")[0]
print(img_url)
type
    #作者
author_name = soup.select("a.name")
author_name = author_name[0].text
mydict['author_name'] = author_name

    #作者 url
author_url = soup.find("a", {"class":"name"})
author_url = author_url.get("href")
mydict['author_url'] = author_url

    # 標籤 再做細部處理==>去掉旁邊的html標籤
tag_list=[]
tags = soup.find_all("a", {"rel":"tag"})
for tag in tags:
    tag = tag.text
    tag_list.append(tag)

mydict['tag']=tag

    #觀看人數
see_soup = soup.select("div.stat")
see_soup = see_soup[0].text
mydict['see']=see_soup

    #餐廳電話號碼
phone_number = soup.select("div.phone.right")[0].text.split(":")[1]
phone_number
mydict['phone_number'] = phone_number
    #超連結
url = soup.find("a", {"target":"_self"})
url= url.get("href")
#     url = soup.select('div.restaurant.item.right > h4 > a')
#     for tag in url:
#         url = tag['href']
url_final = 'https://ifoodie.tw'+url
mydict['web_url'] = url_final
print(mydict)
    

   

https://pic.pimg.tw/huang626162/1488551438-1535622062_l.jpg
{'tag': '216巷美十', 'author_name': '茜的美食記', 'author_url': 'https://ifoodie.tw/user/56079d212756dd184f282124', 'web_url': 'https://ifoodie.tw/', '熱門指數': '5', 'rating': '4.0', 'phone_number': ' 0227710006', 'see': ' 觀看: 1380'}


In [219]:
type(img_url)

str

In [282]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import traceback as tb
import json

res = requests.get('https://ifoodie.tw/restaurant/5875235d2756dd4b7e64057a')
soup = BeautifulSoup(res.text, 'lxml')


    #營業地址
mydict={}
reup = soup.select("span.info_detail")
mydict['address']=reup[0].text
    #營業時間
mydict['opentime']=reup[1].text
    #均消
mydict['avg_consume']=reup[2].text

    #餐廳名字
restaurant_name = soup.select("h1.title")[0]
restaurant_name=restaurant_name.text
mydict['restaurant_name']=restaurant_name
    #餐廳分類
reup1 = soup.select("a.info_detail")

for tag1 in reup1:
    tag1=tag1.text
    mydict['category']=tag1

    #訪客言談 細部處理 去掉旁邊的html標籤

messages = soup.select("div.message > div")
for message in messages:
    message = message.text
#     print(message)
mydict['message'] = message

    
    

山丼 - 快滿出來的牛肉讓人快要忍不住 . 溫泉蛋太犯規 ! 士林店還有自己限定的碳烤豚肉丼口味 ( 自動販賣機點餐 / 無服務費 / 士林捷運站一號出口 )
吸睛度比美味高很多的玫瑰和牛丼-山丼士林店@捷運士林站@中正路
捷運士林站美食推薦｜山丼  Choice牛排做成的豪華牛排丼飯
山丼-肉量超多的燒肉丼飯專賣（士林捷運站）


In [277]:
type(message)

bs4.element.ResultSet

In [260]:
res = requests.get('https://ifoodie.tw/restaurant/58a7096b2756dd06f13c338a')
soup = BeautifulSoup(res.text, 'lxml')

In [261]:
mydict={}
reup = soup.select("span.info_detail")
print(reup)
mydict['address']=reup[0].text
    #營業時間
mydict['opentime']=reup[1].text
    #均消
mydict['avg_consume']=reup[2].text

    #餐廳名字
restaurant_name = soup.select("h1.title")[0]
restaurant_name=restaurant_name.text
mydict['restaurant_name']=restaurant_name
    #餐廳分類
reup1 = soup.select("a.info_detail")

for tag1 in reup1:
    tag1=tag1.text
    mydict['category']=tag1


[<span class="info_detail" data-reactid=".wp0ts3fym8.1.0.1.0.0.1.3.0.2" itemprop="address">404台灣台中市北區漢口路四段375號</span>, <span class="info_detail" data-reactid=".wp0ts3fym8.1.0.1.0.0.1.3.1.2" itemprop="telephone">0422360438</span>, <span class="info_detail" data-reactid=".wp0ts3fym8.1.0.1.0.0.1.3.2.2" datetime="尚無營業時間資訊" itemprop="openingHoursSpecification">尚無營業時間資訊</span>]


In [262]:
print(mydict)

{'restaurant_name': 'KING JUICY', 'avg_consume': '尚無營業時間資訊', 'category': '小吃類', 'address': '404台灣台中市北區漢口路四段375號', 'opentime': '0422360438'}


In [263]:
messages = soup.find_all("div", {"itemprop":"description"})

In [264]:
messages

[<div data-reactid=".wp0ts3fym8.1.0.1.0.1.2.1.$0.1.1.1.1" itemprop="description">台中北區國王級美食。KING juicy。食尚玩家報導燒肉咬蛋吐司、脆皮蛋餅、比臉大國王雞排、無毒檸檬蝶豆花手搖飲宵夜時段同時滿足飢餓的靈魂!</div>,
 <div data-reactid=".wp0ts3fym8.1.0.1.0.1.2.1.$1.1.1.1.1" itemprop="description">KING JUICY 國王的 知名燒肉咬蛋和經典脆皮泡菜蛋餅全天供應想吃就吃 還有萬惡脆皮多汁炸雞 </div>]

In [265]:
messages[0]

<div data-reactid=".wp0ts3fym8.1.0.1.0.1.2.1.$0.1.1.1.1" itemprop="description">台中北區國王級美食。KING juicy。食尚玩家報導燒肉咬蛋吐司、脆皮蛋餅、比臉大國王雞排、無毒檸檬蝶豆花手搖飲宵夜時段同時滿足飢餓的靈魂!</div>

In [266]:
messages[0].text

'台中北區國王級美食。KING juicy。食尚玩家報導燒肉咬蛋吐司、脆皮蛋餅、比臉大國王雞排、無毒檸檬蝶豆花手搖飲宵夜時段同時滿足飢餓的靈魂!'

In [267]:
for message in messages:
    print(message.text

台中北區國王級美食。KING juicy。食尚玩家報導燒肉咬蛋吐司、脆皮蛋餅、比臉大國王雞排、無毒檸檬蝶豆花手搖飲宵夜時段同時滿足飢餓的靈魂!
KING JUICY 國王的 知名燒肉咬蛋和經典脆皮泡菜蛋餅全天供應想吃就吃 還有萬惡脆皮多汁炸雞 


In [269]:

message_mans=soup.select('div.right_wrap > div.user_name > a')
for message_man in message_mans:
    message_man = message_man.text
    mydict['message_man'] = message_man
    
    

In [135]:
message_times=soup.select('div.data-reactid')
#for message_time in message_times:
    
print(message_times)
    

[]


In [138]:
message_times = soup.find_all("span", {"data-reactid":"19cqls9naww"})

In [139]:
message_times

[]

In [132]:
type(message_times)

list